In [1]:
import os
from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ColorClip
from moviepy.video.tools.subtitles import SubtitlesClip
from IPython.display import Video
from open_whisper import preprocess
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing

In [2]:
from moviepy.config import change_settings

In [3]:
change_settings({'IMAGEMAGICK_BINARY': "/mmfs1/home/hvn2002/software/bin/magick"})

In [4]:
def generate_video(audio_file, transcript_file, output_file):
    try:
        audio = AudioFileClip(audio_file)
        generator = lambda text: TextClip(
            text, font="Cantarell-Regular", fontsize=24, color="white"
        )

        if open(transcript_file).read() == "":
            print("Silent segment")
            video = ColorClip(size=(800, 420), color=(0, 0, 0), duration=audio.duration)
            video = video.set_audio(audio)
            video.write_videofile(output_file, fps=24)
        else:
            subtitles = SubtitlesClip(transcript_file, generator).set_position(
                ("center", "bottom")
            )
            video = CompositeVideoClip([subtitles], size=(800, 420))
            video = video.set_audio(audio)
            video.write_videofile(output_file, fps=24)
        return 1
    except Exception as e:
        with open("/mmfs1/gscratch/efml/hvn2002/open_whisper/logs/data/filtering/error_gen_video.txt", "a") as f:
            f.write(f"{audio_file} {transcript_file}\t{e}\n")
        return 0

def parallel_generate_video(args):
    status = generate_video(*args)
    return status

def view_video(output_file):
    return Video(output_file, width=800, height=420)

### Getting a new sample

In [ ]:
transcript_dir = "/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts"
transcript_files = []
for root, dirs, files in os.walk(transcript_dir):
    if "segments" in root:
        for f in os.listdir(root):
            transcript_files.append(os.path.join(root, f))
transcript_files

In [ ]:
rng = np.random.default_rng(42)
sample_t_segs = list(rng.choice(transcript_files, 1000, replace=False))
sample_t_segs

['/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/PU7pLJQku1k/segments/00:03:55.250_00:04:24.289.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/cbi8ZGkGylE/segments/00:12:50.530_00:13:19.240.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/-vsWti5iRac/segments/00:01:23.850_00:01:51.600.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/MG3Hn-PUkiA/segments/00:00:29.890_00:00:58.490.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/CAPcDtk0MtQ/segments/00:12:34.359_00:13:03.579.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/8c3xDfk2bNA/segments/00:12:

In [ ]:
sample_a_segs = [path.replace("transcripts", "audio").replace("srt", "m4a") for path in sample_t_segs]
sample_a_segs

['/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/PU7pLJQku1k/segments/00:03:55.250_00:04:24.289.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/cbi8ZGkGylE/segments/00:12:50.530_00:13:19.240.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/-vsWti5iRac/segments/00:01:23.850_00:01:51.600.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/MG3Hn-PUkiA/segments/00:00:29.890_00:00:58.490.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/CAPcDtk0MtQ/segments/00:12:34.359_00:13:03.579.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/8c3xDfk2bNA/segments/00:12:09.850_00:12:36.800.m4a',
 '/mmfs1/gscratch/efml/hvn20

### Using sample

In [5]:
with open("/mmfs1/gscratch/efml/hvn2002/open_whisper/logs/data/filtering/sample_check.txt", "r") as f:
    sample_t_segs = [line.strip() for line in f]

In [6]:
sample_t_segs

['/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/PU7pLJQku1k/segments/00:03:55.250_00:04:24.289.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/cbi8ZGkGylE/segments/00:12:50.530_00:13:19.240.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/-vsWti5iRac/segments/00:01:23.850_00:01:51.600.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/MG3Hn-PUkiA/segments/00:00:29.890_00:00:58.490.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/CAPcDtk0MtQ/segments/00:12:34.359_00:13:03.579.srt',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/8c3xDfk2bNA/segments/00:12:

In [7]:
sample_a_segs = [path.replace("transcripts", "audio").replace("srt", "m4a") for path in sample_t_segs]

In [8]:
sample_a_segs

['/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/PU7pLJQku1k/segments/00:03:55.250_00:04:24.289.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/cbi8ZGkGylE/segments/00:12:50.530_00:13:19.240.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/-vsWti5iRac/segments/00:01:23.850_00:01:51.600.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/MG3Hn-PUkiA/segments/00:00:29.890_00:00:58.490.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/CAPcDtk0MtQ/segments/00:12:34.359_00:13:03.579.m4a',
 '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/8c3xDfk2bNA/segments/00:12:09.850_00:12:36.800.m4a',
 '/mmfs1/gscratch/efml/hvn20

In [9]:
atv_list = []
for i in range(len(sample_t_segs)):
    original_path = sample_t_segs[i]
    output_path = original_path.replace("transcripts", "filtering").replace("/segments", "").replace("srt", "mp4").replace(":", "_").replace(".", "_", 2)

    os.makedirs(f"/mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/{original_path.split('/')[8]}", exist_ok=True)
    atv_list.append((sample_a_segs[i], original_path, output_path))

In [10]:
atv_list

[('/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.m4a',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/XfVFPVDwrGs/segments/00:40:54.070_00:41:22.330.srt',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/XfVFPVDwrGs/00_40_54_070_00_41_22_330.mp4'),
 ('/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.m4a',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/dRJgDVblxSw/segments/01:20:38.640_01:20:40.640.srt',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/dRJgDVblxSw/01_20_38_640_01_20_40_640.mp4'),
 ('/mmfs1/gscratch/efml/hvn2002/open_whisper/data/audio/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.m4a',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/transcripts/R_nTEmcGlNs/segments/00:05:21.000_00:05:22.500.srt',
  '/mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/R_nTEmcGlNs/00_05_21_000_00_05_22_500.mp4'),
 ('/mmfs1/gsc

### Generating video - only use if need to generate new sample

In [ ]:
with multiprocessing.Pool() as pool:
    results = list(tqdm(pool.imap_unordered(parallel_generate_video, atv_list, chunksize=5), total=len(atv_list)))

Silent segment


Moviepy - Building video /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/1EPJie0LPCg/00_02_53_460_00_02_54_000.mp4.


MoviePy - Writing audio in 00_02_53_460_00_02_54_000TEMP_MPY_wvf_snd.mp3


Moviepy - Building video /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/MlI5fCg3IVo/01_09_37_105_01_09_56_824.mp4.


MoviePy - Writing audio in 01_09_37_105_01_09_56_824TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !


Moviepy - video ready /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/BknUwXpO2nk/00_03_53_755_00_04_22_409.mp4


Moviepy - Done !


Moviepy - video ready /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/vh13CaA3wJA/00_00_59_176_00_01_28_497.mp4


100%|██████████| 1000/1000 [48:17<00:00,  2.90s/it]6it/s, now=None]


In [ ]:
generate_video(*atv_list[0])

chunk:   4%|▍         | 28/624 [02:15<00:12, 47.86it/s, now=None]

Moviepy - Building video /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/XfVFPVDwrGs/00_40_54_070_00_41_22_330.mp4.
MoviePy - Writing audio in 00_40_54_070_00_41_22_330TEMP_MPY_wvf_snd.mp3


chunk:   4%|▍         | 28/624 [02:17<00:12, 47.86it/s, now=None]

MoviePy - Done.
Moviepy - Writing video /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/XfVFPVDwrGs/00_40_54_070_00_41_22_330.mp4



chunk:   4%|▍         | 28/624 [02:20<00:12, 47.86it/s, now=None]

Moviepy - Done !
Moviepy - video ready /mmfs1/gscratch/efml/hvn2002/open_whisper/data/filtering/XfVFPVDwrGs/00_40_54_070_00_41_22_330.mp4


###

### Viewing videos

In [11]:
view_video(atv_list[10][-1])